In [1]:
#!pip install transformers==3.5.0
!pip install torch==1.7.1
!pip install tensorflow==1.4.0

In [10]:
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
import torch
import pandas as pd
import os 
import json
import tensorflow as tf

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/anaconda3/envs/python3

In [11]:
import datetime
import time

In [3]:
print(torch.__version__)

1.7.1


## Model Functions

Loads model from checkpoints and config file  

In [5]:
def load_model(file_path):
    
    model = AutoModelForQuestionAnswering.from_pretrained(file_path)
    
    return model 

In [6]:
def perform_answering(val_df, model, tokenizer):

    all_answers = []
    
    for i, row in val_df.iterrows():
        
        text = row['context']
        question = row['question']
        
        inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt", truncation=True)
        input_ids = inputs["input_ids"].tolist()[0]

        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        answer_start_scores, answer_end_scores = model(**inputs)

        answer_start = torch.argmax(
            answer_start_scores
        )  # Get the most likely beginning of answer with the argmax of the score
        answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        #print(f"Question: {question}")
        #print(f"Answer: {answer}\n")
        
        all_answers.append(answer)
        
    return all_answers
        

In [51]:
def unpack_squad_val(data, col):
    contexts = []
    questions = []
    answers_text = []
    answers_start = []
    for i in range(data.shape[0]):
        topic = data.iloc[i,1]['paragraphs']
        for sub_para in topic:
            for q_a in sub_para['qas']:
                if(q_a['answers']):
                    questions.append(q_a['question'])
                    answers_start.append(q_a['answers'][col]['answer_start'])
                    answers_text.append(q_a['answers'][col]['text'])
                    contexts.append(sub_para['context'])   
    return contexts, questions, answers_text, answers_start 

In [43]:
def find_original_qs(gen_df, val_df):
    
    # find the matching contexts and their questions 
    matches = []
    for idx, rows in gen_df.iterrows():
        inner_matches = []
        string = rows['context']

        for idx, row2 in val_df.iterrows():
            if row2['context'] in string:
                inner_matches.append(row2['question'])

        matches.append(inner_matches)
    
    return matches

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

## Model Metrics

In [99]:
def extract_metrics(filename):

    model_compare = []

    with open(filename,'r') as f:
        lines = f.readlines()
        
        # extract training time 
        start = lines[3][18:]
        end = lines[len(lines)-1][16:]
        start_time = datetime.datetime.strptime(start[:-9], '%y-%m-%d %H:%M:%S')
        end_time = datetime.datetime.strptime(end[:-9], '%y-%m-%d %H:%M:%S')
        train_time = end_time-start_time

        model_params = str(lines[0]).replace("'", '"')
        json_str = str(lines[5]).replace("'", '"')

        #print(json.loads(model_params))
        #print(json.loads(json_str))

        model_compare.append([str(model_params), json.loads(json_str), str(train_time)])
        
    return model_compare

In [100]:
def read_metric_files(directory): 
    model_arr = []
    for file in os.listdir(directory):
        if file.endswith(".txt"): 
            text_name = directory + "/" + file
            model_arr.extend(extract_metrics(text_name))
            
    return model_arr

In [101]:
def comparison_tbl(model_arr):
    
    comparison_tbl = pd.DataFrame(model_arr, columns=["Parameters", "Data", "Train Time"]) 
    
    data_expanded = pd.json_normalize(comparison_tbl['Data'])

    comparison_tbl = pd.concat([comparison_tbl, data_expanded],axis=1)
    
    return comparison_tbl

### Create Metric Files 

In [102]:
model_arr = read_metric_files('results/Hypertuning/SQUAD/Metrics')

In [103]:
metric_table = comparison_tbl(model_arr)

In [106]:
metric_table.to_csv("results/Hypertuning/squad_metrics_V3.csv", sep=",")

In [107]:
model_arr = read_metric_files('results/Hypertuning/QUAC/QUAC Metrics')

In [108]:
metric_table = comparison_tbl(model_arr)

In [109]:
metric_table.to_csv("results/Hypertuning/quac_metrics_V3.csv", sep=",")

## SQuAD Testing

In [12]:
squad_eval_data = "../Data/SQUAD/dev-v2.0.json"

In [13]:
base_squad_trained = load_model('results/Hypertuning/SQUAD/hypertune_8/')

In [14]:
ground_truth_squad = pd.read_json(squad_eval_data) 
    
contexts, questions, answers_text, answers_start = unpack_squad_val(ground_truth_squad, 0)
   
squad_val_data = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})
    

In [25]:
pred = perform_answering(squad_val_data, base_squad_trained, tokenizer)

In [26]:
squad_val_data['pred'] = pred
#squad_val_data.to_csv("csv_pred/hypertune_8.csv", sep=",")

In [27]:
squad_val_data.to_csv("csv_pred/hypertune_8.csv", sep=",")

In [38]:
# getting max length of context 
#val_data.context.map(lambda x: len(x)).max()

## QuAC Training 

In [54]:
quac_eval_data = "../Data/QUAC/val_v0.2_edited.json"

In [55]:
base_quac_trained = load_model('results/Hypertuning/QUAC/hypertune_12/')

In [56]:
ground_truth_quac = pd.read_json(quac_eval_data) 
    
contexts, questions, answers_text, answers_start = unpack_squad_val(ground_truth_quac, 0)
   
quac_val_data = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})


In [57]:
quac_pred = perform_answering(val_data, base_quac_trained, tokenizer)

In [58]:
quac_val_data['pred'] = quac_pred


In [59]:
quac_val_data.to_csv("csv_pred/quac_hypertune_12.csv", sep=",")

# Generated SQuAD Predictions

In [33]:
squad_gen = pd.read_csv('results/squad_gen.csv', index_col=0)
squad_gen.columns = ['context', 'question']

In [34]:
squad_gen

,context,question
0,The Normans (Norman: Nourmands; French: Norman...,What is the first name of the Normans?
1,The Normans (Norman: Nourmands; French: Norman...,What was the first name given to the Normans?
2,The Normans (Norman: Nourmands; French: Norman...,Who was the leader of the Normans?
3,The Normans (Norman: Nourmands; French: Norman...,What was the name of the nation that gave the...
4,The Normans (Norman: Nourmands; French: Norman...,What did the Normans come from?
...,...,...
6628,"The pound-force has a metric counterpart, less...",What is the kilogram-force equivalent to?
6629,"The pound-force has a metric counterpart, less...",What is the mug?
6630,"The pound-force has a metric counterpart, less...",What is the ball?
6631,"The pound-force has a metric counterpart, less...",What is the kip?


In [35]:
squad_gen_pred = perform_answering(squad_gen, base_squad_trained, tokenizer)

In [37]:
squad_gen['pred'] = squad_gen_pred

In [40]:
squad_gen['original questions'] = find_original_qs(squad_gen, squad_val_data)

In [41]:
squad_gen

,context,question,pred,original questions
0,The Normans (Norman: Nourmands; French: Norman...,What is the first name of the Normans?,nourmands,"[In what country is Normandy located?, When we..."
1,The Normans (Norman: Nourmands; French: Norman...,What was the first name given to the Normans?,nourmands,"[In what country is Normandy located?, When we..."
2,The Normans (Norman: Nourmands; French: Norman...,Who was the leader of the Normans?,rollo,"[In what country is Normandy located?, When we..."
3,The Normans (Norman: Nourmands; French: Norman...,What was the name of the nation that gave the...,normans,"[In what country is Normandy located?, When we..."
4,The Normans (Norman: Nourmands; French: Norman...,What did the Normans come from?,"norse ( "" norman "" comes from "" norseman "" ) r...","[In what country is Normandy located?, When we..."
...,...,...,...,...
6628,"The pound-force has a metric counterpart, less...",What is the kilogram-force equivalent to?,[CLS],[What is the metric term less used than the Ne...
6629,"The pound-force has a metric counterpart, less...",What is the mug?,metric slug,[What is the metric term less used than the Ne...
6630,"The pound-force has a metric counterpart, less...",What is the ball?,[CLS],[What is the metric term less used than the Ne...
6631,"The pound-force has a metric counterpart, less...",What is the kip?,1000 lbf,[What is the metric term less used than the Ne...


In [42]:
squad_gen.to_csv('results/squad_gen_qa.csv')

# Generated QuAC Predictions

In [60]:
quac_gen = pd.read_csv('results/quac_gen.csv', index_col=0)
quac_gen.columns = ['context', 'question']

In [61]:
quac_gen

,context,question
0,"In May 1983, she married Nikos Karvelas, a com...",What is 2017?
1,"In September 2016 Vladimir Markin, official sp...",Did the investigation do anything else intere...
2,Graham returned to the WWWF in April 1977 afte...,What was his debut?
3,In the early 1990s US federal agents were inve...,What was he going to do with the WWF?
4,During the aftermath of the murder of Stefan P...,Did he have any ties to the US?
...,...,...
1031,Red worked on new material for Until We Have F...,What is the album about?
1032,"Red started work on their second album, Innoce...",did he tour with any other bands?
1033,"Before 1948, most Palestinian artists were sel...",What was the inspiration for the designs?
1034,"Reubens was born Paul Rubenfeld in Peekskill, ...",None


In [62]:
quac_gen_pred = perform_answering(quac_gen, base_quac_trained, tokenizer)

In [63]:
quac_gen['pred'] = quac_gen_pred

In [64]:
quac_gen['original questions'] = find_original_qs(quac_gen, quac_val_data)

In [65]:
quac_gen

,context,question,pred,original questions
0,"In May 1983, she married Nikos Karvelas, a com...",What is 2017?,cannotanswer,"[what happened in 1983?, did they have any chi..."
1,"In September 2016 Vladimir Markin, official sp...",Did the investigation do anything else intere...,,"[Did they have any clues?, How did they target..."
2,Graham returned to the WWWF in April 1977 afte...,What was his debut?,graham returned to the wwwf in april 1977 afte...,"[Why did he return to the WWWF?, What was his ..."
3,In the early 1990s US federal agents were inve...,What was he going to do with the WWF?,[CLS],"[what disputes did he have?, why?, did he win ..."
4,During the aftermath of the murder of Stefan P...,Did he have any ties to the US?,the game manhunt was linked after the media wr...,"[How was Jack Thompson's related to Manhunt?, ..."
...,...,...,...,...
1031,Red worked on new material for Until We Have F...,What is the album about?,[CLS],"[what happened in 2010?, when was the album re..."
1032,"Red started work on their second album, Innoce...",did he tour with any other bands?,"in march , the band toured with breaking benja...","[When was Innocence & Instinct released?, What..."
1033,"Before 1948, most Palestinian artists were sel...",What was the inspiration for the designs?,cannotanswer,[Are there any other interesting aspects about...
1034,"Reubens was born Paul Rubenfeld in Peekskill, ...",None,[CLS],"[What is Pee wee's playhouse?, when did the sh..."


In [66]:
quac_gen.to_csv('results/quac_gen_qa.csv')

## QuAC Predictions with SQuAD

In [68]:
quac_gen_squad = quac_gen[['context', 'question']].copy()
quac_gen_squad

,context,question
0,"In May 1983, she married Nikos Karvelas, a com...",What is 2017?
1,"In September 2016 Vladimir Markin, official sp...",Did the investigation do anything else intere...
2,Graham returned to the WWWF in April 1977 afte...,What was his debut?
3,In the early 1990s US federal agents were inve...,What was he going to do with the WWF?
4,During the aftermath of the murder of Stefan P...,Did he have any ties to the US?
...,...,...
1031,Red worked on new material for Until We Have F...,What is the album about?
1032,"Red started work on their second album, Innoce...",did he tour with any other bands?
1033,"Before 1948, most Palestinian artists were sel...",What was the inspiration for the designs?
1034,"Reubens was born Paul Rubenfeld in Peekskill, ...",None


In [69]:
quac_squad_pred = perform_answering(quac_gen_squad, base_squad_trained, tokenizer)

In [70]:
quac_gen_squad['pred'] = quac_squad_pred

In [71]:
quac_gen_squad['original questions'] = quac_gen['original questions']

In [72]:
quac_gen_squad

,context,question,pred,original questions
0,"In May 1983, she married Nikos Karvelas, a com...",What is 2017?,[CLS],"[what happened in 1983?, did they have any chi..."
1,"In September 2016 Vladimir Markin, official sp...",Did the investigation do anything else intere...,included the killing of anna politkovskaya amo...,"[Did they have any clues?, How did they target..."
2,Graham returned to the WWWF in April 1977 afte...,What was his debut?,"baltimore , maryland","[Why did he return to the WWWF?, What was his ..."
3,In the early 1990s US federal agents were inve...,What was he going to do with the WWF?,sold his hall of fame ring on ebay to help pay...,"[what disputes did he have?, why?, did he win ..."
4,During the aftermath of the murder of Stefan P...,Did he have any ties to the US?,the police officially denied any link,"[How was Jack Thompson's related to Manhunt?, ..."
...,...,...,...,...
1031,Red worked on new material for Until We Have F...,What is the album about?,novel,"[what happened in 2010?, when was the album re..."
1032,"Red started work on their second album, Innoce...",did he tour with any other bands?,[CLS],"[When was Innocence & Instinct released?, What..."
1033,"Before 1948, most Palestinian artists were sel...",What was the inspiration for the designs?,[CLS],[Are there any other interesting aspects about...
1034,"Reubens was born Paul Rubenfeld in Peekskill, ...",None,[CLS],"[What is Pee wee's playhouse?, when did the sh..."


In [73]:
quac_gen_squad.to_csv('results/quac_squad_gen_qa.csv')